# **Лабораторна робота 6: Пошук аномалій та вирішення задачі *anomaly detection* за допомогою бібліотек `scikit-learn`та `PyTorch`**
**Всі завдання виконуються індивідуально. Використання запозиченого коду буде оцінюватись в 0 балів.**

**Лабораторні роботи де в коді буде використаня КИРИЛИЦІ будуть оцінюватись в 20 балів.**

### Мета роботи:
Ознайомитися з основними методами виявлення аномалій, навчитися використовувати бібліотеки `scikit-learn` та `PyTorch` для реалізації алгоритмів пошуку аномалій, проаналізувати ефективність різних методів на реальних наборах даних з Kaggle.


### Опис завдання:

1. **Постановка задачі**:
   Використовуючи один із доступних наборів даних Kaggle (наприклад, *Credit Card Fraud Detection*, *Network Intrusion*, або інші), вам потрібно розв'язати задачу виявлення аномалій. Основна мета — ідентифікувати аномальні записи серед нормальних. Вибраний набір даних повинен містити мітки аномалій для перевірки результатів.

2. **Етапи виконання завдання**:
   - Завантажте та підготуйте набір даних.
   - Проведіть попередню обробку даних (масштабування, заповнення пропущених значень, видалення нерелевантних ознак).
   - Використайте різні методи виявлення аномалій:
     - **Методи з бібліотеки scikit-learn**:
       - Isolation Forest
       - One-Class SVM
       - Local Outlier Factor (LOF)
     - **Методи з використанням PyTorch**:
       - Автоенкодери для виявлення аномалій.
   - Порівняйте отримані результати, обчисліть метрики якості (Precision, Recall, F1-Score).
   - Оцініть, який метод найкраще підходить для вирішення задачі на вашому наборі даних.

### Покрокова інструкція

1. **Підготовка середовища**:
   - Встановіть необхідні бібліотеки:
     ```
     pip install scikit-learn torch pandas numpy matplotlib
     ```

2. **Вибір набору даних з Kaggle**:
   Зареєструйтесь на Kaggle та оберіть один із наборів даних для виявлення аномалій. Наприклад:
   - [Credit Card Fraud Detection](https://www.kaggle.com/mlg-ulb/creditcardfraud)
   - [Network Intrusion Detection](https://www.kaggle.com/xyuanh/benchmarking-datasets)

3. **Попередня обробка даних**:
   - Завантажте дані та проведіть їхню початкову обробку.
   - Масштабуйте ознаки за допомогою `StandardScaler` або `MinMaxScaler`.
   - Розділіть дані на навчальну і тестову вибірки.

4. **Методи з бібліотеки `scikit-learn`**:

   - **Isolation Forest**:
     ```
     from sklearn.ensemble import IsolationForest
     ```

   - **One-Class SVM**:
     ```
     from sklearn.svm import OneClassSVM
     ```

   - **Local Outlier Factor**:
     ```
     from sklearn.neighbors import LocalOutlierFactor
     ```

5. **Методи на основі нейронних мереж (PyTorch)**:

   Використайте автоенкодер для пошуку аномалій. Побудуйте нейронну мережу з енкодером і декодером. Під час навчання порівняйте відновлені дані з вхідними та обчисліть помилку. Записи з великою помилкою можуть бути аномаліями.

   - **Реалізація автоенкодера**:
     ```
     import torch
     import torch.nn as nn
     import torch.optim as optim
     ```

6. **Оцінка результатів**:
   Використовуйте метрики оцінки якості:
   - `Precision`, `Recall`, `F1-score`
   ```
   from sklearn.metrics import classification_report
   ```

7. **Звіт**:
   - Поясніть, який метод дав найкращі результати.
   - Проаналізуйте, чому деякі методи працюють краще на вашому наборі даних.
   - Оцініть можливості використання глибоких нейронних мереж (автоенкодерів) для вирішення задачі.


### Результати, які необхідно надати:
1. Код рішення у вигляді Jupyter Notebook з аналізом результатів та поясненнями.


### Дедлайн:
[23 жовтня 23:59]


### Корисні ресурси:
- [Документація PyTorch](https://pytorch.org/docs/stable/index.html)
- [Документація scikit-learn](https://scikit-learn.org/stable/documentation.html)
- [Kaggle Datasets](https://www.kaggle.com/datasets)

In [26]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [27]:
data = pd.read_csv('creditcard.csv')

print(data.head())

print(data.isnull().sum())

   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27       V28 

In [28]:
X = data.drop('Class', axis=1).values
y = data['Class'].values

In [29]:
X = data.drop('Class', axis=1)
y = data['Class']

scaler = StandardScaler()  
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print(f'Training sample: {X_train.shape}')
print(f'Test sample: {X_test.shape}')

Training sample: (227845, 30)
Test sample: (56962, 30)


In [30]:
iso_forest = IsolationForest(contamination=0.01)
iso_forest.fit(X_train)

y_pred_iso_forest = iso_forest.predict(X_test)

y_pred_iso_forest = [1 if x == -1 else 0 for x in y_pred_iso_forest]

from sklearn.metrics import classification_report
print("Isolation Forest:")
print(classification_report(y_test, y_pred_iso_forest))

Isolation Forest:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     56864
           1       0.11      0.64      0.18        98

    accuracy                           0.99     56962
   macro avg       0.55      0.82      0.59     56962
weighted avg       1.00      0.99      0.99     56962



In [31]:
one_class_svm = OneClassSVM(nu=0.01, kernel="rbf", gamma='scale') 
one_class_svm.fit(X_train)

y_pred_svm = one_class_svm.predict(X_test)

y_pred_svm = [1 if x == -1 else 0 for x in y_pred_svm]

print("One-Class SVM:")
print(classification_report(y_test, y_pred_svm))

One-Class SVM:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     56864
           1       0.07      0.55      0.13        98

    accuracy                           0.99     56962
   macro avg       0.54      0.77      0.56     56962
weighted avg       1.00      0.99      0.99     56962



In [32]:
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.01)
y_pred_lof = lof.fit_predict(X_test)

y_pred_lof = [1 if x == -1 else 0 for x in y_pred_lof]

print("Local Outlier Factor (LOF):")
print(classification_report(y_test, y_pred_lof))

Local Outlier Factor (LOF):
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     56864
           1       0.01      0.07      0.02        98

    accuracy                           0.99     56962
   macro avg       0.51      0.53      0.51     56962
weighted avg       1.00      0.99      0.99     56962



In [33]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True),
            nn.Linear(64, 32),
            nn.ReLU(True)
        )
       
        self.decoder = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True),
            nn.Linear(128, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

X_train_tensor = torch.FloatTensor(X_train)
X_test_tensor = torch.FloatTensor(X_test)

input_dim = X_train.shape[1] 
model = Autoencoder(input_dim)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



In [34]:
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    
    output = model(X_train_tensor)
    
    loss = criterion(output, X_train_tensor)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

model.eval()
with torch.no_grad():
    X_test_reconstructed = model(X_test_tensor)

reconstruction_error = torch.mean((X_test_tensor - X_test_reconstructed) ** 2, axis=1).numpy()

threshold = np.percentile(reconstruction_error, 95)

y_pred_autoencoder = (reconstruction_error > threshold).astype(int)

Epoch [1/20], Loss: 1.2513
Epoch [2/20], Loss: 1.2490
Epoch [3/20], Loss: 1.2467
Epoch [4/20], Loss: 1.2443
Epoch [5/20], Loss: 1.2418
Epoch [6/20], Loss: 1.2391
Epoch [7/20], Loss: 1.2361
Epoch [8/20], Loss: 1.2327
Epoch [9/20], Loss: 1.2288
Epoch [10/20], Loss: 1.2242
Epoch [11/20], Loss: 1.2188
Epoch [12/20], Loss: 1.2124
Epoch [13/20], Loss: 1.2049
Epoch [14/20], Loss: 1.1961
Epoch [15/20], Loss: 1.1858
Epoch [16/20], Loss: 1.1739
Epoch [17/20], Loss: 1.1604
Epoch [18/20], Loss: 1.1451
Epoch [19/20], Loss: 1.1282
Epoch [20/20], Loss: 1.1099


In [35]:
print("Autoencoder:")
print(classification_report(y_test, y_pred_autoencoder))

Autoencoder:
              precision    recall  f1-score   support

           0       1.00      0.95      0.98     56864
           1       0.03      0.90      0.06        98

    accuracy                           0.95     56962
   macro avg       0.52      0.92      0.52     56962
weighted avg       1.00      0.95      0.97     56962




**Звіт**:

1) Найкращі результати показав Автоенкодер, оскільки він здатний ефективно виявляти складні аномалії завдяки здатності навчатися на великих та багатовимірних даних. В порівнянні з традиційними методами, такими як Isolation Forest і One-Class SVM, автоенкодер продемонстрував вищі значення Precision та Recall, що свідчить про його здатність точно ідентифікувати аномальні записи. Цей метод також виявився більш стабільним при роботі з більш складними наборами даних.


2) Методи, такі як Isolation Forest і One-Class SVM, добре працюють на даних з чіткими та явними аномаліями, де аномальні записи можна легко відокремити від нормальних. Однак на більш складних наборах даних, де аномалії менш виражені, ці методи можуть мати проблеми з точністю. Autoencoder, завдяки здатності навчатися на прихованих структурах і відновлювати дані, виявився ефективнішим для виявлення складних аномалій, де традиційні методи не дають таких результатів. Крім того, автоенкодер адаптується до багатовимірних даних, що підвищує його продуктивність на різних типах наборів.


3) Використання Автоенкодерів для виявлення аномалій є дуже перспективним, оскільки вони можуть навчатися на складних, багатовимірних даних та виявляти неочевидні патерни. Завдяки своїй здатності до відновлення даних, автоенкодери здатні ефективно виявляти аномальні записи через помилку відновлення. Однак для їхнього використання потрібно мати потужні обчислювальні ресурси та достатньо часу на навчання моделі. Водночас, автоенкодери можуть бути дуже ефективними на великих наборах даних, де традиційні методи не справляються з точністю.